In [20]:
import cv2
import hyperspy.api as hs
import kornia.feature as KF
import kornia as K
import numpy as np
import torch
import matplotlib.pyplot as plt
from kornia_moons.viz import *
from kornia_moons.feature import *
%matplotlib qt5

In [ ]:
fname1 = "kn_church-2.jpg"
fname2 = "kn_church-8.jpg"

img0 = K.io.load_image(fname1, K.io.ImageLoadType.RGB32)[None]
img1 = K.io.load_image(fname2, K.io.ImageLoadType.RGB32)[None]
img0 = K.geometry.resize(img0, (600, 375), antialias=True)
img1 = K.geometry.resize(img1, (600, 375), antialias=True)

input_dict = {
    "image0": K.color.rgb_to_grayscale(img0),  # LofTR works on grayscale images only
    "image1": K.color.rgb_to_grayscale(img1),
}

In [3]:
def numpy_to_torch_tensor(npy):
    npy = npy - np.amin(npy)
    npy = 255*(npy/np.amax(npy))
    tensor = np.zeros((1,1,np.shape(npy)[0],np.shape(npy)[1]))
    tensor[0,0] = npy
    tensor = torch.FloatTensor(tensor)
    return tensor

In [4]:
img0 = numpy_to_torch_tensor(np.load(r"C:\Users\tas72\Documents\PhD\dg606\20230624\Stitching_and_corr\test\20230624_100334_vbf.npy"))
img1 = numpy_to_torch_tensor(np.load(r"C:\Users\tas72\Documents\PhD\dg606\20230624\Stitching_and_corr\test\20230624_100824_vbf.npy"))
input_dict = {"image0":img0,
             "image1":img1}

In [5]:
fig,ax = plt.subplots()
ax.imshow(K.tensor_to_image(img1))

In [11]:
matcher = KF.LoFTR(pretrained="indoor")

In [17]:
with torch.inference_mode():
    correspondences = matcher(input_dict)

AttributeError: 'dict' object has no attribute 'shape'

In [13]:
for k, v in correspondences.items():
    print(k)

keypoints0
keypoints1
confidence
batch_indexes


In [14]:
mkpts0 = correspondences["keypoints0"].cpu().numpy()
mkpts1 = correspondences["keypoints1"].cpu().numpy()

In [15]:
mkpts0

array([], shape=(0, 2), dtype=float32)

In [84]:
Fm, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
inliers = inliers > 0


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\usac\ransac_solvers.cpp:1046: error: (-2:Unspecified error) in function 'auto __cdecl cv::usac::mergePoints::<lambda_1ca39094813444db980908f76b22a0b5>::operator ()(class cv::Mat &,int) const'
> Invalid dimension of point (expected: 'points.cols >= pt_dim'), where
>     'points.cols' is 0
> must be greater than or equal to
>     'pt_dim' is 2


In [71]:
draw_LAF_matches(
    KF.laf_from_center_scale_ori(
        torch.from_numpy(mkpts0).view(1, -1, 2),
        torch.ones(mkpts0.shape[0]).view(1, -1, 1, 1),
        torch.ones(mkpts0.shape[0]).view(1, -1, 1),
    ),
    KF.laf_from_center_scale_ori(
        torch.from_numpy(mkpts1).view(1, -1, 2),
        torch.ones(mkpts1.shape[0]).view(1, -1, 1, 1),
        torch.ones(mkpts1.shape[0]).view(1, -1, 1),
    ),
    torch.arange(mkpts0.shape[0]).view(-1, 1).repeat(1, 2),
    K.tensor_to_image(img0),
    K.tensor_to_image(img1),
    inliers,
    draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": None, "feature_color": (0.2, 0.5, 1), "vertical": False},
)

Using SIFT and concerting it manually

In [45]:
def get_matching_kpts(lafs1, lafs2, idxs):
    src_pts = KF.get_laf_center(lafs1).view(-1, 2)[idxs[:, 0]].detach().cpu().numpy()
    dst_pts = KF.get_laf_center(lafs2).view(-1, 2)[idxs[:, 1]].detach().cpu().numpy()
    return src_pts, dst_pts
    
def sift_matching(fname1, fname2):
    img1 = cv2.cvtColor(cv2.imread(fname1), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(fname2), cv2.COLOR_BGR2RGB)
    
    # OpenCV SIFT
    sift = cv2.SIFT_create(8000)
    kps1, descs1 = sift.detectAndCompute(img1, None)
    kps2, descs2 = sift.detectAndCompute(img2, None)

    # Converting to kornia for matching via AdaLAM
    lafs1 = laf_from_opencv_SIFT_kpts(kps1)
    lafs2 = laf_from_opencv_SIFT_kpts(kps2)
    dists, idxs = KF.match_adalam(
        torch.from_numpy(descs1), torch.from_numpy(descs2), lafs1, lafs2, hw1=img1.shape[:2], hw2=img2.shape[:2]
    )

    # Converting back to kornia via to use OpenCV MAGSAC++
    tentatives = cv2_matches_from_kornia(dists, idxs)
    src_pts = np.float32([kps1[m.queryIdx].pt for m in tentatives]).reshape(-1, 2)
    dst_pts = np.float32([kps2[m.trainIdx].pt for m in tentatives]).reshape(-1, 2)

    F, inliers_mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.USAC_MAGSAC, 0.25, 0.999, 100000)

    # Drawing matches using kornia_moons
    draw_LAF_matches(
        lafs1,
        lafs2,
        idxs,
        img1,
        img2,
        inliers_mask,
        draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": None, "feature_color": None, "vertical": False},
    )
    print(f"{inliers_mask.sum()} inliers found")
    return

In [46]:
fname1 = "kn_church-2.jpg"
fname2 = "kn_church-8.jpg"
sift_matching(fname1, fname2)

11 inliers found


In [47]:
def get_matching_kpts(lafs1, lafs2, idxs):
    src_pts = KF.get_laf_center(lafs1).view(-1, 2)[idxs[:, 0]].detach().cpu().numpy()
    dst_pts = KF.get_laf_center(lafs2).view(-1, 2)[idxs[:, 1]].detach().cpu().numpy()
    return src_pts, dst_pts


def sift_korniadesc_matching(fname1, fname2, descriptor):
    timg1 = K.io.load_image(fname1, K.io.ImageLoadType.RGB32)[None, ...]  # BxCxHxW
    timg2 = K.io.load_image(fname2, K.io.ImageLoadType.RGB32)[None, ...]  # BxCxHxW

    sift = OpenCVDetectorKornia(cv2.SIFT_create(8000))
    local_feature = KF.LocalFeature(sift, KF.LAFDescriptor(descriptor))

    lafs1, resps1, descs1 = local_feature(K.color.rgb_to_grayscale(timg1))
    lafs2, resps2, descs2 = local_feature(K.color.rgb_to_grayscale(timg2))

    dists, idxs = KF.match_adalam(descs1[0], descs2[0], lafs1, lafs2, hw1=timg1.shape[2:], hw2=timg2.shape[2:])

    src_pts, dst_pts = get_matching_kpts(lafs1, lafs2, idxs)
    F, inliers_mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.USAC_MAGSAC, 0.25, 0.999, 100000)
    draw_LAF_matches(
        lafs1,
        lafs2,
        idxs,
        K.tensor_to_image(timg1),
        K.tensor_to_image(timg2),
        inliers_mask,
        draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": None, "feature_color": None, "vertical": False},
    )
    print(f"{inliers_mask.sum()} inliers found")

In [48]:
mkd = KF.MKDDescriptor(32)
with torch.inference_mode():
    sift_korniadesc_matching(fname1, fname2, mkd)

15 inliers found


In [27]:
tfeat = KF.TFeat(True)
with torch.inference_mode():
    sift_korniadesc_matching(fname1, fname2, tfeat)

Downloading: "https://github.com/vbalnt/tfeat/raw/master/pretrained-models/tfeat-liberty.params" to C:\Users\tas72/.cache\torch\hub\checkpoints\tfeat-liberty.params
100%|█████████████████████████████████████████████████████████████████████████████| 2.29M/2.29M [00:00<00:00, 85.9MB/s]


18 inliers found


In [28]:
device = torch.device("cpu")
hardnet = KF.HardNet(True).eval()
with torch.inference_mode():
    sift_korniadesc_matching(fname1, fname2, hardnet)

26 inliers found


In [34]:
def siftaffnet_korniadesc_matching(fname1, fname2, descriptor):
    timg1 = K.io.load_image(fname1, K.io.ImageLoadType.RGB32)[None, ...]  # BxCxHxW
    timg2 = K.io.load_image(fname2, K.io.ImageLoadType.RGB32)[None, ...]  # BxCxHxW

    # Magic is here
    sift = OpenCVDetectorWithAffNetKornia(cv2.SIFT_create(8000))

    local_feature = KF.LocalFeature(sift, KF.LAFDescriptor(descriptor))
    with torch.inference_mode():
        lafs1, resps1, descs1 = local_feature(K.color.rgb_to_grayscale(timg1))
        lafs2, resps2, descs2 = local_feature(K.color.rgb_to_grayscale(timg2))
        dists, idxs = KF.match_adalam(descs1[0], descs2[0], lafs1, lafs2, hw1=timg1.shape[2:], hw2=timg2.shape[2:])

    src_pts, dst_pts = get_matching_kpts(lafs1, lafs2, idxs)

    F, inliers_mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.USAC_MAGSAC, 0.25, 0.999, 100000)
    print(F)
    draw_LAF_matches(
        lafs1,
        lafs2,
        idxs,
        K.tensor_to_image(timg1),
        K.tensor_to_image(timg2),
        inliers_mask,
        draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": None, "feature_color": None, "vertical": False},
    )
    print(f"{inliers_mask.sum()} inliers found")

In [35]:
siftaffnet_korniadesc_matching(fname1, fname2, hardnet)


[[ 4.50187827e-07 -2.31419734e-06  1.85316246e-03]
 [-6.50406627e-06 -1.31747622e-06  1.06966896e-02]
 [ 5.24452351e-03 -4.69812817e-03 -2.41414730e+00]]
34 inliers found


In [36]:
siftaffnet_korniadesc_matching(fname1, fname2, KF.HyNet(True).eval())

[[-4.85230503e-07 -9.18246490e-07  1.37376044e-03]
 [-8.98271988e-06 -1.64258660e-06  1.13717952e-02]
 [ 6.41669411e-03 -4.91509637e-03 -2.54892365e+00]]
47 inliers found


In [37]:
siftaffnet_korniadesc_matching(fname1, fname2, KF.SOSNet(True).eval())

[[ 1.02526735e-06  2.51399907e-07 -1.20717426e-03]
 [ 1.05014194e-05  1.48716450e-06 -1.14522576e-02]
 [-6.95772479e-03  5.00638956e-03  2.56243873e+00]]
47 inliers found


In [38]:
siftaffnet_korniadesc_matching(fname1, fname2, KF.HardNet8(True).eval())

[[ 2.55459316e-07  2.04939806e-07 -4.38493907e-04]
 [ 3.96477866e-06  5.18799795e-07 -4.43864472e-03]
 [-2.74114707e-03  1.96796122e-03  1.00000000e+00]]
37 inliers found


In [49]:
# %%capture
fname1 = "kn_church-2.jpg"
fname2 = "kn_church-8.jpg"

adalam_config = KF.adalam.get_adalam_default_config()
# adalam_config['orientation_difference_threshold'] = None
# adalam_config['scale_rate_threshold'] = None
adalam_config["force_seed_mnn"] = False
adalam_config["search_expansion"] = 16
adalam_config["ransac_iters"] = 256


img1 = K.io.load_image(fname1, K.io.ImageLoadType.RGB32, device=device)[None, ...]
img2 = K.io.load_image(fname2, K.io.ImageLoadType.RGB32, device=device)[None, ...]

num_features = 2048
disk = KF.DISK.from_pretrained("depth").to(device)

hw1 = torch.tensor(img1.shape[2:], device=device)
hw2 = torch.tensor(img2.shape[2:], device=device)

match_with_adalam = True

with torch.inference_mode():
    inp = torch.cat([img1, img2], dim=0)
    features1, features2 = disk(inp, num_features, pad_if_not_divisible=True)
    kps1, descs1 = features1.keypoints, features1.descriptors
    kps2, descs2 = features2.keypoints, features2.descriptors
    if match_with_adalam:
        lafs1 = KF.laf_from_center_scale_ori(kps1[None], 96 * torch.ones(1, len(kps1), 1, 1, device=device))
        lafs2 = KF.laf_from_center_scale_ori(kps2[None], 96 * torch.ones(1, len(kps2), 1, 1, device=device))

        dists, idxs = KF.match_adalam(descs1, descs2, lafs1, lafs2, hw1=hw1, hw2=hw2, config=adalam_config)
    else:
        dists, idxs = KF.match_smnn(descs1, descs2, 0.98)


print(f"{idxs.shape[0]} tentative matches with DISK AdaLAM")

Downloading: "https://raw.githubusercontent.com/cvlab-epfl/disk/master/depth-save.pth" to C:\Users\tas72/.cache\torch\hub\checkpoints\depth-save.pth
100%|█████████████████████████████████████████████████████████████████████████████| 4.17M/4.17M [00:00<00:00, 95.1MB/s]


222 tentative matches with DISK AdaLAM


In [50]:
def get_matching_keypoints(kp1, kp2, idxs):
    mkpts1 = kp1[idxs[:, 0]]
    mkpts2 = kp2[idxs[:, 1]]
    return mkpts1, mkpts2


mkpts1, mkpts2 = get_matching_keypoints(kps1, kps2, idxs)

Fm, inliers = cv2.findFundamentalMat(
    mkpts1.detach().cpu().numpy(), mkpts2.detach().cpu().numpy(), cv2.USAC_MAGSAC, 1.0, 0.999, 100000
)
inliers = inliers > 0
print(f"{inliers.sum()} inliers with DISK")

99 inliers with DISK


In [52]:
draw_LAF_matches(
    KF.laf_from_center_scale_ori(kps1[None].cpu()),
    KF.laf_from_center_scale_ori(kps2[None].cpu()),
    idxs.cpu(),
    K.tensor_to_image(img1.cpu()),
    K.tensor_to_image(img2.cpu()),
    inliers,
    draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": (1, 1, 0.2, 0.3), "feature_color": None, "vertical": False},
)

In [ ]:
get_matching_keypoints()